In [1]:
# sa walker - extract supplementary alignment regions from SA tags

In [153]:
import os
import sys

# args
print(sys.argv)
if "sa_walker" in sys.argv[0] or "stdin" in sys.argv[0]:
    if len(sys.argv) != 6:
        print("usage: " + sys.argv[0] + " <input-cram> <in-bed> <ref-fasta> <edge-threshold> <out-bed>\n")
        sys.exit(-1)
    # commandline invocation
    IN_CRAM = sys.argv[1]
    IN_BED = sys.argv[2]
    REF_FASTA = sys.argv[3]
    EDGE_THRESHOLD = int(sys.argv[4])
    OUT_BED = sys.argv[5]
else:
    IN_CRAM = os.path.expanduser("~/tmp/data/jump_align/chr22.cram")
    IN_BED = "data/chr22_all.bed"
    REF_FASTA = os.path.expanduser("~/tmp/ref/Homo_sapiens_assembly38.fasta")
    EDGE_THRESHOLD = 3
    OUT_BED = "/tmp/sa_walker." + str(os.getpid()) + ".bed"

QUANTIZE=20

['/Users/drorkessler/miniconda3/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/Users/drorkessler/Library/Jupyter/runtime/kernel-e007ece3-9170-44e0-a0c7-ff9469929940.json']


In [144]:
from itertools import groupby

def query_len(cigar_string):
    read_consuming_ops = ("M", "I", "S", "=", "X")
    result = 0
    cig_iter = groupby(cigar_string, lambda chr: chr.isdigit())
    for _, length_digits in cig_iter:
        length = int(''.join(length_digits))
        op = next(next(cig_iter)[1])
        if op in read_consuming_ops:
            result += length
    return result

In [157]:
import networkx as nx
graph = nx.DiGraph()

def process_gap(lchrom, lloc, rchrom, rloc):
    #print("process_gap", [lchrom, lloc, rchrom, rloc])
    if not "_" in lchrom + rchrom:
        lnode = (lchrom, lloc - lloc % QUANTIZE + int(QUANTIZE / 2))
        rnode = (rchrom, rloc - rloc % QUANTIZE + int(QUANTIZE / 2))
        if not graph.has_node(lnode):
            graph.add_node(lnode)
        if not graph.has_node(rnode):
            graph.add_node(rnode)
        if graph.has_edge(lnode, rnode):
            data = graph.get_edge_data(lnode, rnode)
            data["c"] = data["c"] + 1
        else:
            graph.add_edge(lnode, rnode, c=1)


In [158]:

def process_sa(read):

    # current alignment represents the left side of the gap
    lchrom = read.reference_name
    lloc = read.reference_end

    sa_all = read.get_tag("SA").split(";")
    if sa_all[-1] == '':
        sa_all = sa_all[:-1]
    #print("sa_all", sa_all)
    for sa_index in range(len(sa_all)):
        toks = sa_all[sa_index].split(",")

        # establish right side of gap
        #print("sa_index", sa_index, "toks", toks)
        rchrom = toks[0]
        rloc = int(toks[1])

        # register this gap
        process_gap(lchrom, lloc, rchrom, rloc)

        # establish next left side, if there is one
        if sa_index + 1 < len(sa_all):
            lchrom = rchrom
            lloc = rloc + query_len(toks[3])
    

In [159]:
import pysam

with pysam.AlignmentFile(IN_CRAM, "rb", reference_filename=REF_FASTA) as samf:
    
    # loop on bed regions
    read_count = 0
    supp_count = 0
    with open(IN_BED) as f:
        for line in f:
            bed_line = line.strip().split()
            chrom, start, end = bed_line[:3]
            start = int(start)
            end = int(end)

            # loop on reads
            for read in samf.fetch(chrom, start, end):
                read_count += 1
                if read.is_supplementary:
                    supp_count += 1
                    process_sa(read)

                if read_count % 1000000 == 0:
                    print("read_count", read_count, "supp_count", supp_count, "read", read.reference_name + ":" + str(read.reference_start))
                    sys.stdout.flush()



read_count 1000000 supp_count 75914 read chr22:18160110
read_count 2000000 supp_count 92395 read chr22:25215724
read_count 3000000 supp_count 104880 read chr22:32152302
read_count 4000000 supp_count 117092 read chr22:39096855
read_count 5000000 supp_count 130211 read chr22:46172271


In [173]:
# collect potential gaps
gaps = []
for edge in graph.edges:
    c = graph.edges[edge[0], edge[1]]["c"]
    if c > 2:
        gaps.append([edge[0], edge[1], c])

# filter on threshold
gaps = [x for x in gaps if x[2] >= EDGE_THRESHOLD]
print("len(gaps)", len(gaps))

# sort of left edge
def sort_key(chrom):
    if chrom == "chrX":
        return 23
    elif chrom == "chrY":
        return 24
    else:
        return int(chrom[3:])
    
gaps = sorted(gaps, key=lambda x: sort_key(x[0][0]))
for gap in gaps[:5]:
    print(gap)

# write to output file
print("OUT_BED", OUT_BED)
with open(OUT_BED, "w") as f:
    for gap in gaps:
        f.write("%s\t%d\t%s\t%d\t%d\n" % (gap[0][0], gap[0][1], gap[1][0], gap[1][1], gap[2]))

len(gaps) 3854
[('chr1', 224015030), ('chr10', 42320530), 3]
[('chr1', 23884670), ('chr1', 90846950), 4]
[('chr1', 167056830), ('chr7', 157299690), 3]
[('chr2', 89840550), ('chr20', 31073230), 3]
[('chr2', 89836810), ('chr2', 89836470), 3]
OUT_BED /tmp/sa_walker.97585.bed
